Link: https://github.com/run-llama/llama_index/tree/main

In [ ]:
%pip install llama-index-llms-huggingface
%pip install llama-index

In [2]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
!pip install langchain_community
!pip install pypdf

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader

In [4]:
!mkdir data

In [3]:
def loadDocs(folderPath : str):
    loader = DirectoryLoader(
        path = folderPath,
        glob= "*.pdf",
        loader_cls=PyPDFLoader,
    )
    docs = loader.load()
    return docs

In [4]:
docs = loadDocs("/content/data")
print(len(docs))
print(docs[0])

63
page_content='KEY DEFINITIONS
Household income: Includes income of the 
householder and all other people 15 years and 
older in the household, whether or not they are 
related to the householder.
Median: The point that divides the household 
income distribution into halves, one half with 
income above the median and the other with 
income below the median. The median is based 
on the income distribution of all households, 
including those with no income.
Gini index: A summary measure of income 
inequality. The Gini index varies from 0 to 1, with 
0 indicating perfect equality, where there is a 
proportional distribution of income. A Gini index 
of 1 indicates perfect inequality, where one 
household has all the income.
Household Income in States and 
Metropolitan Areas: 2022
American Community Survey Briefs
By Kirby G. Posey
ACSBR-017
December 2023
INTRODUCTION
This brief presents statistics on median household 
income and the Gini index of income inequality based 
on the 2022 (and 

In [6]:
from llama_index.core import PromptTemplate

systemPrompt = """<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

queryWrapperPrompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM

llm = HuggingFaceLLM(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto",
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=systemPrompt,
    query_wrapper_prompt=queryWrapperPrompt,
    tokenizer_kwargs={"max_length": 2048},
    model_kwargs={"torch_dtype": torch.float16}
)


In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model =HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.core.settings import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024

##### LangChain Document objects and LlamaIndex Document objects look similar but are not compatible because they are designed for different internal pipelines and data models. A LangChain Document only contains page_content and metadata, while LlamaIndex requires each document to have a unique id_ field and a .text attribute for indexing, chunk tracking, and graph construction. When LlamaIndex builds a VectorStoreIndex, it assumes every document already has this internal identity and schema; LangChain documents do not, so LlamaIndex fails when it tries to access doc.id_. This is not a bug but a design boundary: LangChain focuses on orchestration and retrieval abstractions, while LlamaIndex focuses on structured indexing and query engines. Therefore, LangChain documents must be explicitly converted into LlamaIndex documents before indexing, and the two cannot be used interchangeably at the object level.

In [6]:
documents = SimpleDirectoryReader("./data/").load_data()

In [18]:
index = VectorStoreIndex.from_documents(documents)

In [19]:
query_engine = index.as_query_engine()

In [ ]:
query_engine.query("What are thr Health Insurance Coverage Status and Type by Geography")